# New Parsing Dev

In [1]:
from cayley_dickson_base import CayleyDicksonBase
from cayley_dickson_integers import Zi
from cayley_dickson_rationals import Qi
import generic_utils as utils

import re

## Test Quaternion String Parsing

In [2]:
from unittest import TestCase, TextTestRunner, defaultTestLoader
from cayley_dickson_integers import Zi
from random import seed

class TestZi(TestCase):

    def setUp(self) -> None:
        pass

    def test_parse_hypercomplex_string(self):
        self.assertEqual(Zi.parse_hypercomplex_string('1+2i+3j+4k'), [1, 2, 3, 4])
        self.assertEqual(Zi.parse_hypercomplex_string('-1+3i-3j+7k'), [-1, 3, -3, 7])
        self.assertEqual(Zi.parse_hypercomplex_string('-1-4i-9j-2k'), [-1, -4, -9, -2])
        self.assertEqual(Zi.parse_hypercomplex_string('17-16i-15j-14k'), [17, -16, -15, -14])
        self.assertEqual(Zi.parse_hypercomplex_string('7+2i'), [7, 2])
        self.assertEqual(Zi.parse_hypercomplex_string('7+2j'), [7, 0, 2, 0])
        self.assertEqual(Zi.parse_hypercomplex_string('2i-6k'), [0, 2, 0, -6])
        self.assertEqual(Zi.parse_hypercomplex_string('1-5j+2k'), [1, 0, -5, 2])
        self.assertEqual(Zi.parse_hypercomplex_string('1-5j'), [1, 0, -5, 0])
        self.assertEqual(Zi.parse_hypercomplex_string('3+4i-9k'), [3, 4, 0, -9])
        self.assertEqual(Zi.parse_hypercomplex_string('42i+j-k'), [0, 42, 1, -1])
        self.assertEqual(Zi.parse_hypercomplex_string('6-2i+j-3k'), [6, -2, 1, -3])
        self.assertEqual(Zi.parse_hypercomplex_string('1+i+j+k'), [1, 1, 1, 1])
        self.assertEqual(Zi.parse_hypercomplex_string('-1-i-j-k'), [-1, -1, -1, -1])
        self.assertEqual(Zi.parse_hypercomplex_string('16k-20j+2i-7'), [-7, 2, -20, 16])
        self.assertEqual(Zi.parse_hypercomplex_string('i+4k-3j+2'), [2, 1, -3, 4])
        self.assertEqual(Zi.parse_hypercomplex_string('5k-2i+9+3j'), [9, -2, 3, 5])
        self.assertEqual(Zi.parse_hypercomplex_string('5k-2j+3'), [3, 0, -2, 5])
        self.assertEqual(Zi.parse_hypercomplex_string('1.75-1.75i-1.75j-1.75k'), [1.75, -1.75, -1.75, -1.75])
        self.assertEqual(Zi.parse_hypercomplex_string('2.0j-3k+0.47i-13'), [-13, 0.47, 2.0, -3])
        self.assertEqual(Zi.parse_hypercomplex_string('5.6-3i'), [5.6, -3])
        self.assertEqual(Zi.parse_hypercomplex_string('k-7.6i'), [0, -7.6, 0, 1])
        self.assertEqual(Zi.parse_hypercomplex_string('0'), [0, 0])
        self.assertEqual(Zi.parse_hypercomplex_string('0j+0k'), [0, 0])
        self.assertEqual(Zi.parse_hypercomplex_string('-0j'), [0, 0])
        self.assertEqual(Zi.parse_hypercomplex_string('1-0k'), [1, 0])
        self.assertEqual(Zi.parse_hypercomplex_string('1+2i+3j+4k'), [1, 2, 3, 4])
        self.assertEqual(Zi.parse_hypercomplex_string('7.1E-2 +4.3k+i'), [0.071, 1, 0, 4.3])
        self.assertEqual(Zi.parse_hypercomplex_string('3 - 2E-3i-4j'), [3, -0.002, -4, 0])

In [3]:
TextTestRunner(verbosity=2).run(defaultTestLoader.loadTestsFromTestCase(TestZi))

test_parse_hypercomplex_string (__main__.TestZi.test_parse_hypercomplex_string) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [4]:
qstrings = ['2i+3j+4k-1.07+6I+7J+8K',
            '-0j',
            '-1 + 0j +0k - i',
            '1+2i+3j+4k+5L+6I+7J+8K',
            '3.14159 + 0.4j + 4k  - 2e3i ',
            '4k-3j-2i+1',
            '2i - 4 + k',
            '2i + 4 - k',
            'k',
            'i+j']

In [5]:
for q in qstrings:
    print(f"{q} --> {Zi.parse_hypercomplex_string(q)}\n")

2i+3j+4k-1.07+6I+7J+8K --> [-1.07, 2, 3, 4, 0, 6, 7, 8]

-0j --> [0, 0]

-1 + 0j +0k - i --> [-1, -1]

1+2i+3j+4k+5L+6I+7J+8K --> [1, 2, 3, 4, 5, 6, 7, 8]

3.14159 + 0.4j + 4k  - 2e3i  --> [3.14159, -2000, 0.4, 4]

4k-3j-2i+1 --> [1, -2, -3, 4]

2i - 4 + k --> [-4, 2, 0, 1]

2i + 4 - k --> [4, 2, 0, -1]

k --> [0, 0, 0, 1]

i+j --> [0, 1, 1, 0]



## Code Golf Parser

See this website (near the bottom) regarding the Code Golf program below:

https://codegolf.stackexchange.com/questions/76545/parse-a-quaternion

In [20]:
from re import sub, findall

def w(r):

    # Substitute all minus (-) and plus (+) signs NOT followed by a number  (if there are any) with a "-1"/"+1",
    # respectively.
    a=sub('[+](?![0-9])','+1',sub('[-](?![0-9])','-1',r))

    # Lambda function created for later use to sort the Quaternion. This function, when given as a key to the 
    # "sorted" function, arranges the input Quaternion in the order where the whole number comes first, and
    # then the rest are placed in order of increasing letter value (i,j,k in this case) 
    q=lambda x:(not x.isdigit(),''.join(filter(str.isalpha,x)))

    # The following "for" loop replaces the letters NOT preceded by a number with a one followed by that letter
    for z in findall('(?<![0-9])[a-z]',a):
        a=a.replace(z,('+1{}'.format(z)))

    # The following first substitutes all pluses and minuses (+ and -) with a space, and then that new string
    # is split at those spaces, and returned as a list. After that, the list is sorted according the the
    # "lambda" function shown above. Then, the first item in that list, which is supposed to be a lone number,
    # is checked to make sure that it indeed is a lone number. If it isn't, then "+0, " is appended to the
    # Quaternion. 
    if not str(sorted(((sub('[.]','',sub('[+-]',' ',a))).split(' ')),key=q)[0]).isdigit():
        a+='+0, '

    # The following "for" loop finds ALL the letters NOT in the list, by finding the symmetric difference
    # between a set of all the letters found, and a set containing all the letters needed. For the letters
    # not in the list, a '+0' is added the quaternion, followed by that letter, and then a comma and a space.
    for i in list(set(findall('[a-z]',a))^{'i','j','k'}):
        a+='+0{}, '.format(i)

    # Finally, in this last step, a ", " is added IN BETWEEN unicode characters and pluses/minuses (+/-).
    # Then, it splits at those spaces, and the commas separate different parts of the Quaternion from
    # each other (otherwise, you would get something like `12i+3j+4k` from `2i+3j+4k+1`) in a returned list.
    # Then, that list is sorted according to the lambda expression "q" (above), and then, finally, the NUMBERS
    # (of any type, courtesy to Regex) are extracted from that joined list, and printed out in the correct order.
    print(findall('[-]?\d+(?:\.\d+)?',''.join(sorted(sub('(?<=[A-Za-z0-9])(?=[+-])',', ',a).split(' '),key=q))))

In [21]:
q_parser_tests = [

    # Test String              Expected Result
    #-------------------------------------------------------------------
    ("1+2i+3j+4k",             [1, 2, 3, 4]),
    ("-1+3i-3j+7k",            [-1, 3, -3, 7]),
    ("-1-4i-9j-2k",            [-1, -4, -9, -2]),
    ("17-16i-15j-14k",         [17, -16, -15, -14]),
    
    ("7+2i",                    [7, 2, 0, 0]),
    ("2i-6k",                   [0, 2, 0, -6]),
    ("1-5j+2k",                 [1, 0, -5, 2]),
    ("3+4i-9k",                 [3, 4, 0, -9]),
    
    ("42i+j-k",                 [0, 42, 1, -1]),
    ("6-2i+j-3k",               [6, -2, 1, -3]),
    ("1+i+j+k",                 [1, 1, 1, 1]),
    ("-1-i-j-k",                [-1, -1, -1, -1]),

    ("16k-20j+2i-7",            [-7, 2, -20, 16]),
    ("i+4k-3j+2",               [2, 1, -3, 4]),
    ("5k-2i+9+3j",              [9, -2, 3, 5]),
    ("5k-2j+3",                 [3, 0, -2, 5]),
    
    ("1.75-1.75i-1.75j-1.75k",  [1.75, -1.75, -1.75, -1.75]),
    ("2.0j-3k+0.47i-13",        [-13, 0.47, 2.0, -3]),  # or [-13 .47 2 -3]
    ("5.6-3i",                  [5.6, -3, 0, 0]),
    ("k-7.6i",                  [0, -7.6, 0, 1]),
    
    ("0",                       [0, 0, 0, 0]),
    ("0j+0k",                   [0, 0, 0, 0]),
    ("-0j",                     [0, 0, 0, 0]), # or [0 0 -0 0]
    ("1-0k",                    [1, 0, 0, 0]),  # or [1 0 0 -0]
    
    ("1+2i+3j+4K",              [1, 2, 3, 4]), # Uses K instead of k
    ("7.1E-2 +4.3k+i",          [0.071, 1, 0, 4.3]),  # scientific notation
    ("3 - 2E-3i-4j",            [3, -0.002, -4, 0])  # more sci notation
]

In [24]:
for test in q_parser_tests:
    print(f"{w(test[0])}\n")

1+2i+3j+4k
1+2i+3j+4k

['1', '2', '3', '4']
None

-1+3i-3j+7k
-1+3i-3j+7k

['-1', '3', '-3', '7']
None

-1-4i-9j-2k
-1-4i-9j-2k

['-1', '-4', '-9', '-2']
None

17-16i-15j-14k
17-16i-15j-14k

['17', '-16', '-15', '-14']
None

7+2i
7+2i

['7', '2', '0', '0']
None

2i-6k
2i-6k

['0', '2', '0', '-6']
None

1-5j+2k
1-5j+2k

['1', '0', '-5', '2']
None

3+4i-9k
3+4i-9k

['3', '4', '0', '-9']
None

42i+j-k
42i+1j-1k

['0', '42', '1', '-1']
None

6-2i+j-3k
6-2i+1j-3k

['6', '-2', '1', '-3']
None

1+i+j+k
1+1i+1j+1k

['1', '1', '1', '1']
None

-1-i-j-k
-1-1i-1j-1k

['-1', '-1', '-1', '-1']
None

16k-20j+2i-7
16k-20j+2i-7

['-7', '2', '-20', '16']
None

i+4k-3j+2
i+4k-3j+2

['2', '1', '-3', '4']
None

5k-2i+9+3j
5k-2i+9+3j

['9', '-2', '3', '5']
None

5k-2j+3
5k-2j+3

['3', '0', '-2', '5']
None

1.75-1.75i-1.75j-1.75k
1.75-1.75i-1.75j-1.75k

['1.75', '-1.75', '-1.75', '-1.75']
None

2.0j-3k+0.47i-13
2.0j-3k+0.47i-13

['-13', '0.47', '2.0', '-3']
None

5.6-3i
5.6-3i

['5.6', '-3', '0', '0']
None

